# MNE : From raw data to epochs and evoked responses (ERF/ERP)

`
Authors:
Alexandre Gramfort
Denis A. Engemann
`

In [1]:
%matplotlib qt
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

First, load the _mne_ package:

In [1]:
import mne

C:\Users\Usuario\Anaconda3\lib\site-packages\numba\decorators.py:146: RuntimeWarning: Caching is not available when the 'parallel' target is in use. Caching is now being disabled to allow execution to continue.
  warnings.warn(msg, RuntimeWarning)


We set the log-level to 'warning' so the output is less verbose

In [ ]:
mne.set_log_level('warning')

### Remember if you need help just ask... the machine

In [ ]:
mne.pick_types?

## Access raw data

You should have downloaded the `ds000117-practical` folder.

In [ ]:
import os

# Change the following path to where the folder ds000117-practical is on your disk
data_path = os.path.expanduser("~/work/data/ds000117-practical/")

raw_fname = os.path.join(data_path,
    'derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg.fif')

In [ ]:
print(raw_fname)

In [ ]:
ls $raw_fname

Read data from file:

In [ ]:
mne.io.read_raw_fif?

In [ ]:
raw = mne.io.read_raw_fif(raw_fname, preload=False)
print(raw)

Note the `preload=False` which states that no data is actually in memory.

For general info on importing MEG see:
https://mne.tools/stable/auto_tutorials/io/plot_10_reading_meg_data.html
or EEG see:
https://mne.tools/stable/auto_tutorials/io/plot_20_reading_eeg_data.html

Now let's look at the measurement info. It will give details about:

   - sampling rate
   - filtering parameters
   - available channel types
   - bad channels
   - etc.

In [2]:
mne.__version__

'0.19.2'

In [3]:
mne.sys_info()

Platform:      Windows-10-10.0.18362-SP0
Python:        3.7.1 (default, Dec 10 2018, 22:54:23) [MSC v.1915 64 bit (AMD64)]
Executable:    C:\Users\Usuario\Anaconda3\python.exe
CPU:           Intel64 Family 6 Model 158 Stepping 10, GenuineIntel: 12 cores
Memory:        31.9 GB

mne:           0.19.2
numpy:         1.16.1 {blas=C:\\projects\\numpy-wheels\\numpy\\build\\openblas, lapack=C:\\projects\\numpy-wheels\\numpy\\build\\openblas}
scipy:         1.1.0
matplotlib:    3.0.2 {backend=module://ipykernel.pylab.backend_inline}

sklearn:       0.20.1
numba:         0.41.0
nibabel:       2.5.1
cupy:          Not found
pandas:        0.23.4
dipy:          1.0.0
mayavi:        4.7.1 {qt_api=pyqt5, PyQt5=5.9.2}
pyvista:       Not found
vtk:           8.2.0


In [ ]:
print(raw.info)

<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>How many channels do you have for each type of sensors?</li>
    <li>What is the sampling frequency?</li>
    <li>Have the data been filtered?</li>
    <li>What is the frequency of the line noise?</li>
    <li>Is there any bad channel?</li>
    </ul>
</div>

INSERT ANSWERS HERE

raw.info is just a dictionary:

In [ ]:
isinstance(raw.info, dict)

So we can access its elements this way:

In [ ]:
raw.info['sfreq']  # Sampling frequency

In [ ]:
raw.info['bads']  # list of marked bad channels

In [ ]:
raw.info['line_freq']

Next let's see what channels are present. It is available via the `raw.ch_names` attribute.

In [ ]:
type(raw.ch_names)

In [ ]:
raw.ch_names[:10]

You can index it as a list

In [ ]:
raw.ch_names[42]

In [ ]:
raw.ch_names[:3]

Channel type of a specific channel

In [ ]:
channel_type = mne.io.pick.channel_type(raw.info, 75)
print('Channel #75 is of type:', channel_type)

channel_type = mne.io.pick.channel_type(raw.info, 320)
print('Channel #320 is of type:', channel_type)

Info contains all the details about the sensors (type, locations, coordinate frame etc.)

In [ ]:
len(raw.info['chs'])

In [ ]:
type(raw.info['chs'])

In [ ]:
raw.info['chs'][0]

In [ ]:
raw.info['chs'][330]

In [ ]:
raw.plot_sensors(kind='topomap', ch_type='grad');

In [ ]:
raw.plot_sensors(kind='topomap', ch_type='eeg');

In [ ]:
raw.info

### Setting channel types

Some channels are wrongly defined as EEG in the file. 2 of these are EOG (EEG061 and EEG062) and EEG063 is actually an ECG channel. EEG064 was recording but not connected to anything, so we'll not make it `'misc'`. We will now set the channel types. This will be useful for automatic artifact rejection.

In [ ]:
raw.set_channel_types?

In [ ]:
raw.set_channel_types({'EEG061': 'eog',
                       'EEG062': 'eog',
                       'EEG063': 'ecg',
                       'EEG064': 'misc'})  # EEG064 free-floating el.

raw.rename_channels({'EEG061': 'EOG061',
                     'EEG062': 'EOG062',
                     'EEG063': 'ECG063'})

In [ ]:
raw.info

In [ ]:
raw.plot_sensors(kind='topomap', ch_type='eeg');

## Accessing the data

To access the data just use the [] syntax as to access any element of a list, dict etc.

In [ ]:
start, stop = 0, 10
data, times = raw[:, start:stop]  # fetch all channels and the first 10 time points
print(data.shape)
print(times.shape)

In [ ]:
times  # always starts at 0 by convention

Note that `raw[]` returns both the data and the times array.

# Resampling the data

We will now change the sampling frequency of the data to speed up the computations.

In [ ]:
raw.load_data()  # it is required to load data in memory
raw.resample(300)

And let's remove the unecessary channels

In [ ]:
raw.drop_channels?

In [ ]:
to_drop = ['STI201', 'STI301', 'MISC201', 'MISC202', 'MISC203',
           'MISC204', 'MISC205', 'MISC206', 'MISC301', 'MISC302',
           'MISC303', 'MISC304', 'MISC305', 'MISC306', 'CHPI001',
           'CHPI002', 'CHPI003', 'CHPI004', 'CHPI005', 'CHPI006',
           'CHPI007', 'CHPI008', 'CHPI009']

In [ ]:
raw.drop_channels(to_drop)

In [ ]:
raw.info

# Visualizing raw data

See https://mne.tools/0.16/auto_tutorials/plot_visualize_raw.html
for more details.

Let's look at how to:
- browse data
- turn On/Off the PCA/SSP projections
- mark bad segments to obtained annotations
- group channel by types
- group channel by location

In [ ]:
raw.plot?

In [ ]:
%matplotlib qt

raw.plot();

<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>Do you see any bad channel?</li>
    <li>Do you see any bad segment of data?</li>
    <li>Do you see any more then EOG blinks?</li>
    </ul>
</div>

In [ ]:
raw.annotations

In [ ]:
raw.annotations.save('annot.csv')

In [ ]:
!cat annot.csv

### Filtering

In [ ]:
raw.filter?

<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>Filter the raw data between 0Hz and 40Hz.</li>
    </ul>
</div>

In [ ]:
raw.filter(0, 40)

In [ ]:
raw.filter?

<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>Plot the 10 first seconds of stimulation channel just using matplotlib.</li>
    </ul>
</div>

Tips:

- Pick the stim channel using `mne.pick_types`
- Get the data for this channel
- Plot it using `plt.plot`

In [ ]:
# d = raw.get_data(picks=['EEG001', 'EEG002'])
# d = raw.get_data(picks=('eeg', 'mag'))
d = raw.get_data(picks=('grad',))
np.max(d)

In [ ]:
#### TODO
start = 0
stop = int(50 * raw.info['sfreq'])
data = raw.get_data('STI101', start=start, stop=stop)
data.shape

In [ ]:
raw.times[start:stop].shape

In [ ]:
plt.plot(raw.times[start:stop], data.T)

## Define and read epochs

First extract events:

In [ ]:
events = mne.find_events(raw, stim_channel='STI101', verbose=True)

<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>What is the type of the variable events?</li>
    <li>What is the meaning of the 3 columnes of events?</li>
    <li>How many events of type 5 do you see?</li>
    </ul>
</div>

In [ ]:
# events(:, 3) == 5 # matlab
np.sum(events[:, 2] == 5)

There was a time offset of 34.5ms in the stimulus presentation. We need to correct events accordingly.

In [ ]:
delay = int(round(0.0345 * raw.info['sfreq']))
events[:, 0] = events[:, 0] + delay

Let's visualize the paradigm:

In [ ]:
events = events[events[:, 2] < 20] # take only events with code less than 20

In [ ]:
fig = mne.viz.plot_events(events, raw.info['sfreq']);

For event trigger and conditions we use a Python dictionary with keys that contain "/" for grouping sub-conditions

In [ ]:
event_id = {
    'face/famous/first': 5,
    'face/famous/immediate': 6,
    'face/famous/long': 7,
    'face/unfamiliar/first': 13,
    'face/unfamiliar/immediate': 14,
    'face/unfamiliar/long': 15,
    'scrambled/first': 17,
    'scrambled/immediate': 18,
    'scrambled/long': 19,
}

In [ ]:
fig = mne.viz.plot_events(events, sfreq=raw.info['sfreq'],
                          event_id=event_id);

In [ ]:
raw.plot(event_id=event_id, events=events);

Define epochs parameters:

In [ ]:
tmin = -0.5  # start of each epoch (500ms before the trigger)
tmax = 2.0  # end of each epoch (2000ms after the trigger)

Define the baseline period:

In [ ]:
baseline = (-0.2, 0)  # means from 200ms before to stim onset (t = 0)

Define peak-to-peak (amplitude range) rejection parameters for gradiometers, magnetometers and EOG:

In [ ]:
reject = dict(grad=4000e-13, mag=4e-12, eog=150e-6)  # this can be highly data dependent

<div class="alert alert-info">
    <b>REMARK</b>:
     <ul>
    <li>The <a href="https://autoreject.github.io/">autoreject</a> project aims to solve this problem of reject parameter setting. See the <a href="https://www.sciencedirect.com/science/article/pii/S1053811917305013">paper</a>.</li>
    </ul>
</div>

In [ ]:
# we are picky again, this time with EOG
picks = mne.pick_types(raw.info, meg=True, eeg=True, eog=True,
                       stim=False, exclude='bads')

In [ ]:
picks

Extract epochs:

In [ ]:
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True,
                    picks=picks, baseline=baseline,
                    reject=reject)

In [ ]:
print(epochs)

In [ ]:
epochs.drop_bad()  # remove bad epochs based on reject

In [ ]:
epochs.load_data()  # load data in memory

Explore the epochs namespace

Hit ``epochs.<TAB>``

In [ ]:
# epochs.

See how epochs were dropped

In [ ]:
epochs.plot_drop_log();

In [ ]:
for drop_log in epochs.drop_log[:20]:
    print(drop_log)

In [ ]:
epochs.copy().drop(10, reason="I don't like this one").plot_drop_log();

In [ ]:
epochs.copy().drop(10, reason="I don't like this one").drop_log[:20]

In [ ]:
raw.filter(0, 80)

In [ ]:
events.shape

In [ ]:
epochs.events.shape

In [ ]:
events[epochs.selection] == epochs.events

### Wait a second, did we just loose half of our epochs due to EOG???

We can probably do better. Let's use the PCA-based signal space projection (SSP) to regress out spatial patterns related to EOG and other offenders, ie., ECG.

Here is the workflow, we'll first detect EOG artifacts and visualize their impact. Then we'll compute related spatial patterns to mitigate these artifacts.

In [ ]:
# We can use a convenience function
eog_epochs = mne.preprocessing.create_eog_epochs(raw.copy().filter(1, None))
eog_epochs.average().plot_joint()

In [ ]:
raw.plot(events=eog_epochs.events);

Compare the y axes to the ERF/ERPs we just saw. We face important degrees of contamination!

In [ ]:
projs_eog, _ = mne.preprocessing.compute_proj_eog(
    raw, n_mag=3, n_grad=3, n_eeg=3, average=True)

In [ ]:
projs_eog

In [ ]:
layouts = [mne.find_layout(raw.info, ch_type=ch) for ch in ("eeg", "mag", "grad")]
mne.viz.plot_projs_topomap(projs_eog, layout=layouts);

Now the important question is how many components one should keep? Pro-tip: some of them don't look like clear artifact patterns. The good news is that we don't need to decide __*right*__ now.

In [ ]:
# same business, same issue for ECG
ecg_epochs = mne.preprocessing.create_ecg_epochs(raw.copy().filter(1, None))
ecg_epochs.average().plot_joint()

We also face important insults from the cardiac signal... we'll project that out.

In [ ]:
projs_ecg, _ = mne.preprocessing.compute_proj_ecg(
    raw, n_mag=3, n_grad=3, n_eeg=3, average=True)
mne.viz.plot_projs_topomap(projs_ecg, layout=layouts);

In [ ]:
# now let's see how that would theoretically improve data preservation
reject2 = dict(mag=reject['mag'], grad=reject['grad']) 

epochs_clean = mne.Epochs(raw, events, event_id, tmin, tmax, proj=False,
                          picks=picks, baseline=baseline,
                          preload=False,
                          reject=reject2)

epochs_clean.add_proj(projs_eog + projs_ecg)
#epochs_clean.copy().apply_proj().average().plot(spatial_colors=True);  # apply projs on a copy

In [ ]:
epochs_clean.copy().average().plot(proj='interactive', spatial_colors=True);  # apply projs on a copy

now we keep all trials, probably we also removed some good signals.
we will postpone the selection of SSP vectors to later study the impact on
source localization

<div class="alert alert-info">
    <b>REMARK</b>:
     <ul>
    <li>MNE keeps SSP projections inside the info and allows to apply them later.</li>
    </ul>
</div>

In [ ]:
# let's overwrite
epochs = epochs_clean

<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>Use ICA instead of SSP to remove artifacts</li>
    <li>What are potential benefits or disadvantages?</li>
    </ul>
</div>

### Visualization Epochs

See [this page](https://mne.tools/stable/auto_tutorials/epochs/plot_visualize_epochs.html) for options on how to visualize epochs.

Here is just an illustration to make a so-called ERP/ERF image:

In [ ]:
raw.plot_psd(fmax=40);

In [ ]:
epochs.plot_image(picks='EEG065', sigma=1.);

In [ ]:
import matplotlib.pyplot as plt
plt.close('all')

In [ ]:
epochs.plot();

### The epochs object is your MNE swiss army knife for processing segmented data!

- specialized methods for diagnostic plotting of data
- averaging
- saving
- manipulating data, e.g., rearranging or deleting single trials, resampling

<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>How could you get the epochs corresponding to face?</li>
    <li>How could you get the epochs corresponding to a familiar face?</li>
    <li>How could you get the epochs corresponding to a scrambled face?</li>
    </ul>
</div>

In [ ]:
epochs.event_id

In [ ]:
epochs[['unfamiliar', 'scrambled']]

## basic IO 

The standard scenario is saving the epochs into .fif file together with all the header data.

In [ ]:
epochs_fname = raw_fname.replace('_meg.fif', '-epo.fif')
epochs_fname

In [ ]:
epochs.save(epochs_fname, overwrite=True)  # note that epochs are save in files ending with -epo.fif

In [ ]:
data = epochs.get_data()
data.shape

Scipy also supports reading and writing of matlab files. You can save your single trials with:

In [ ]:
from scipy import io
epochs_data = epochs.get_data()
print(epochs_data.shape)
io.savemat('epochs_data.mat', dict(epochs_data=epochs_data),
           oned_as='row')

## Average the epochs to get ERF/ERP and plot it!

In [ ]:
# refresh evoked
evoked = epochs.average()
evoked.del_proj()  # delete previous proj
# take first for each sensor type
evoked.add_proj(projs_eog[::3] + projs_ecg[::3])
evoked.apply_proj()  # apply

In [ ]:
#evoked = epochs.average()
print(evoked)

In [ ]:
plt.close('all')
evoked.plot(proj=True);

We can also show sensor position as line color:

In [ ]:
evoked.plot(spatial_colors=True, proj=True);  # note the legend

In [ ]:
times = [0.0, 0.1, 0.18]
evoked.plot_topomap(ch_type='mag', times=times, proj=True);
evoked.plot_topomap(ch_type='grad', times=times, proj=True);
evoked.plot_topomap(ch_type='eeg', times=times, proj=True);

In [ ]:
import numpy as np
# pure topography plots called topomap in the MNE jargon
for ch_type in ('mag', 'grad', 'eeg'):
    evoked.plot_topomap(times=np.linspace(0.05, 0.45, 8),
                        ch_type=ch_type, proj=True);

<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>How does SSP impact the evoked responses? Use proj="interactive" to explore</li>
    </ul>
</div>

In [ ]:
# For example, let's say that we want to keep the first projs for now

# refresh evoked
evoked = epochs.average()
evoked.del_proj()  # delete previous proj
# take first for each sensor type
evoked.add_proj(projs_eog[::3] + projs_ecg[::3])
evoked.apply_proj()  # apply

Topoplot and time series can also be shown in one single plot:

In [ ]:
evoked.plot_joint(times=[0.17]);

## Accessing and indexing epochs by condition

Epochs can be indexed by integers or slices to select a subset of epochs but also with strings to select by conditions `epochs[condition]`

In [ ]:
epochs[0]  # first epoch

In [ ]:
epochs[:10]  # first 10 epochs

In [ ]:
epochs['face']  # epochs for a face

In event_id, "/" selects conditions in a hierarchical way, e.g. here, "face" vs. "scrambled", "famous" vs. "unfamiliar", and MNE can select them individually

In [ ]:
epochs['face'].average().\
    pick_types(meg='grad').crop(-0.1, 0.25).plot(spatial_colors=True);

Apply this to visualize all the conditions in `event_id`

In [ ]:
plt.close('all')
for condition in ['face', 'scrambled']:
    epochs[condition].average().plot_topomap(times=[0.1, 0.15], title=condition);

## Write evoked data to disk

In [ ]:
evoked_fname = raw_fname.replace('_meg.fif', '-ave.fif')
evoked_fname

In [ ]:
evoked.save(evoked_fname)  # note that the file for evoked ends with -ave.fif

or to write multiple conditions in 1 file

In [ ]:
evokeds_list = [epochs[k].average() for k in event_id]  # get evokeds
mne.write_evokeds(evoked_fname, evokeds_list)

### Reading evoked from disk

It is also possible to read evoked data stored in a fif file:

In [ ]:
evokeds_list = mne.read_evokeds(evoked_fname, baseline=(None, 0), proj=True)

Or give the explicit name of the averaged condition:

In [ ]:
evoked1 = mne.read_evokeds(evoked_fname, condition="face/famous/first",
                           baseline=(None, 0), proj=True)

**Remark:** Did you notice that you can apply some preprocessing on reading the evokeds from disk?

### Compute a contrast:

In [ ]:
evoked_face = epochs['face'].average()
evoked_scrambled = epochs['scrambled'].average()

In [ ]:
contrast = mne.combine_evoked([evoked_face, evoked_scrambled], [0.5, -0.5])

Note that this combines evokeds taking into account the number of averaged epochs (to scale the noise variance)

In [ ]:
print(evoked1.nave)  # average of 12 epochs
print(contrast.nave)  # average of 116 epochs

In [ ]:
print(contrast)

In [ ]:
fig = contrast.copy().pick('grad').crop(-0.1, 0.3).plot_joint()

In [ ]:
evoked_face

In [ ]:
evoked_scrambled

### Save your figure as pdf

In [ ]:
%matplotlib qt
import numpy as np
contrast.plot_topomap(times=np.linspace(0.05, 0.15, 5), ch_type='mag')
plt.savefig('toto.pdf')
!open toto.pdf  # works only on a mac

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>Compute the evoked data for 'famous', 'unfamiliar', 'scrambled' faces</li>
      <li>Crop the data between -0.2s and 0.4s</li>     
      <li>Plot the channel EEG065 in all 3 conditions using mne.viz.plot_compare_evokeds function</li>
    </ul>
</div>

See: https://mne.tools/stable/generated/mne.viz.plot_compare_evokeds.html

In [ ]:
evoked_famous = epochs['famous'].average().crop(-0.1, 0.4)
evoked_scrambled = epochs['scrambled'].average().crop(-0.1, 0.4)
evoked_unfamiliar = epochs['unfamiliar'].average().crop(-0.1, 0.4)

In [ ]:
plt.close('all')
mne.viz.plot_evoked_topo([evoked_famous, evoked_scrambled, evoked_unfamiliar]);

In [ ]:
evokeds = {k:epochs[k].average().crop(-0.1, 0.4)
           for k in ['famous', 'unfamiliar', 'scrambled']}

In [ ]:
plt.close('all')
mne.viz.plot_compare_evokeds(evokeds, picks='EEG065');

## ADVANCED: Customize your plots

Want to have every text in blue?

In [ ]:
import matplotlib as mpl
fig = evoked1.plot(show=False)  # butterfly plots
fig.subplots_adjust(hspace=1.0)
for text in fig.findobj(mpl.text.Text):
    text.set_fontsize(18)
    text.set_color('blue')
for ax in fig.get_axes():
    ax.axvline(0., color='red', linestyle='--')
plt.tight_layout()
fig.savefig('plot_erf.pdf');